<a href="https://colab.research.google.com/github/ariahosseini/DeepML/blob/main/022_ALIGNN_Proj_TwentyTwo_SuperConductor_CriticalTemp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U jarvis-tools --quiet
!pip install xformers==v0.0.22 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 976.2/976.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.7/807.7 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 19.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.6/211.6 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.8 MB/

In [ ]:
!pip install --upgrade git+https://gitlab.com/ase/ase.git@master

  Cloning https://gitlab.com/ase/ase.git (to revision master) to /tmp/pip-req-build-r6mzx47a
  Running command git clone --filter=blob:none --quiet https://gitlab.com/ase/ase.git /tmp/pip-req-build-r6mzx47a
  Resolved https://gitlab.com/ase/ase.git to commit 28a0a1f1988e3f06c2c887139103c573b74a066c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ase: filename=ase-3.23.0b1-py3-none-any.whl size=2699941 sha256=202751567d63526c455992bb001c291461612eb0ce06d569164118991c76c56a
  Stored in directory: /tmp/pip-ephem-wheel-cache-e46ofu9_/wheels/01/fc/60/e6134958a4ba6921161034c849be512a068712df0013719724
Successfully built ase


In [ ]:
%%time
!pip install -q dgl==1.0.1+cu117 -f https://data.dgl.ai/wheels/cu117/repo.html
!pip install -q alignn phonopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 12.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.1 MB/s eta 0:00:00
CPU times: user 760 ms, sys: 71.3 ms, total: 831 ms
Wall time: 1min 10s


In [ ]:
# utils
import os, sys, itertools, requests, zipfile, io, tempfile, glob, time
import json
import pandas as pd
import numpy as np
# ase
from ase import Atom, Atoms as AseAtoms
from ase import data
# torch
import torch
# jarvis
from jarvis.core.atoms import Atoms
from jarvis.analysis.diffraction.xrd import XRD
from jarvis.io.vasp.inputs import Poscar
from jarvis.io.vasp.outputs import Vasprun
from jarvis.db.figshare import get_jid_data, data as jdata
from jarvis.db.jsonutils import dumpjson, loadjson
from jarvis.core.atoms import ase_to_atoms
# alignn
from alignn.models.alignn import ALIGNN, ALIGNNConfig
from alignn.models.alignn_atomwise import ALIGNNAtomWise, ALIGNNAtomWiseConfig
from alignn.graphs import Graph
from alignn.ff.ff import (phonons, ForceField, AlignnAtomwiseCalculator,
                          default_path, wt10_path,fd_path, alignnff_fmult)
# vis
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!mkdir SuperConductor_CriticalTemp
!cd SuperConductor_CriticalTemp
os.chdir("SuperConductor_CriticalTemp")

In [ ]:
data = {
    "version": "112bbedebdaecf59fb18e11c929080fb2f358246",
    "dataset": "user_data",
    "target": "target",
    "atom_features": "cgcnn",
    "neighbor_strategy": "k-nearest",
    "id_tag": "jid",
    "random_seed": 123,
    "classification_threshold": None,
    "n_val": None,
    "n_test": None,
    "n_train": None,
    "train_ratio": 0.8,
    "val_ratio": 0.1,
    "test_ratio": 0.1,
    "target_multiplication_factor": None,
    "epochs": 3,
    "batch_size": 2,
    "weight_decay": 1e-05,
    "learning_rate": 0.001,
    "filename": "sample",
    "warmup_steps": 2000,
    "criterion": "mse",
    "optimizer": "adamw",
    "scheduler": "onecycle",
    "pin_memory": False,
    "save_dataloader": False,
    "write_checkpoint": True,
    "write_predictions": True,
    "store_outputs": True,
    "progress": True,
    "log_tensorboard": False,
    "standard_scalar_and_pca": False,
    "use_canonize": True,
    "num_workers": 0,
    "cutoff": 8.0,
    "max_neighbors": 12,
    "keep_data_order": False,
    "model": {
        "name": "alignn",
        "alignn_layers": 4,
        "gcn_layers": 4,
        "atom_input_features": 92,
        "edge_input_features": 80,
        "triplet_input_features": 40,
        "embedding_features": 64,
        "hidden_features": 256,
        "output_features": 1,
        "link": "identity",
        "zero_inflated": False,
        "classification": False
    }
}
file_path = "config.json"
with open(file_path, "w") as json_file:
    json.dump(data, json_file)

In [ ]:
outdir = "./"
dft_3d = jdata("dft_3d")
prop = "Tc_supercon"  # "optb88vdw_bandgap"
# max_samples = 50
f = open("id_prop.csv", "w")
count = 0
for i in dft_3d:
    atoms = Atoms.from_dict(i["atoms"])
    jid = i["jid"]
    poscar_name = "POSCAR-" + jid + ".vasp"
    target = i[prop]
    if target != "na":
        atoms.write_poscar(poscar_name)
        f.write("%s,%6f\n" % (poscar_name, target))
        # count += 1
        # if count == max_samples:
        #    break
f.close()

Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699
Loading the zipfile...
Loading completed.


In [ ]:
!ls /usr/local/bin/*alignn*

/usr/local/bin/run_alignn_ff.py  /usr/local/bin/train_alignn.py


In [ ]:
%%time
!train_alignn.py --root_dir "./" --config "config.json" --output_dir=temp

fatal: not a git repository (or any of the parent directories): .git
id_prop_csv_file exists True
Traceback (most recent call last):
  File "/usr/local/bin/train_alignn.py", line 393, in <module>
    train_for_folder(
  File "/usr/local/bin/train_alignn.py", line 177, in train_for_folder
    if config.model.calculate_gradient and config.model.gradwise_weight != 0:
  File "/usr/local/lib/python3.10/dist-packages/pydantic/main.py", line 767, in __getattr__
    raise AttributeError(f'{type(self).__name__!r} object has no attribute {item!r}')
AttributeError: 'ALIGNNConfig' object has no attribute 'calculate_gradient'
CPU times: user 81.2 ms, sys: 11.1 ms, total: 92.3 ms
Wall time: 7.96 s


In [ ]:
data = pd.read_csv("temp/prediction_results_test_set.csv", skiprows=1)
x = data.iloc[:, 1]
y = data.iloc[:, 2]
line_values = np.linspace(min(x.min(), y.min()), max(x.max(), y.max()), 100)
plt.plot(line_values, line_values, label="x=y Reference", color="r", linestyle="--")
plt.scatter(x, y)
plt.xlabel("Target DFT Critical Temp (K)")
plt.ylabel("Predicted ALIGNN (Tc)")
plt.title("Prediction Results for Test Set")
plt.show()

In [ ]:
data = pd.read_csv("temp/prediction_results_train_set.csv", skiprows=1)
x = data.iloc[:, 0]
y = data.iloc[:, 1]
line_values = np.linspace(min(x.min(), y.min()), max(x.max(), y.max()), 100)
plt.plot(line_values, line_values, label="x=y Reference", color="r", linestyle="--")
plt.scatter(x, y)
plt.xlabel("Target DFT Critical Temp (K)")
plt.ylabel("Predicted ALIGNN (Tc)")
plt.title("Prediction Results for Training Set")
plt.show()

In [ ]:
output_features =  1
filename = 'temp/checkpoint_3.pt'
device = "cpu"
if torch.cuda.is_available():
    device = torch.device("cuda")
model = ALIGNN(ALIGNNConfig(name="alignn", output_features=output_features))
model.load_state_dict(torch.load(filename, map_location=device)["model"])
model.eval()

In [ ]:
dft_3d = data('dft_3d')
df=pd.DataFrame(dft_3d)
print(df.columns, len(df.columns))

In [ ]:
df1 = df[['optb88vdw_bandgap','magmom_outcar', 'ehull', 'atoms', 'jid']].replace('na',np.nan).dropna()

In [ ]:
def has_element(atoms_dict = {}, my_element = 'H'):
    atoms = Atoms.from_dict(atoms_dict)
    elements = atoms.elements
    if my_element in elements:
        return True
    else:
        return np.nan
df1['el'] = df1['atoms'].apply(lambda x: has_element(atoms_dict = x, my_element = 'H'))
df1['formula'] = df1['atoms'].apply(lambda x: Atoms.from_dict(x).composition.reduced_formula)

In [ ]:
df2 = df1.dropna()

In [ ]:
df2[(df2['optb88vdw_bandgap'] <= .1) & (df2['ehull'] <= .2) & (df2['magmom_outcar'] <= 0.05)]

In [ ]:
dat = get_jid_data(jid="JVASP-157789", dataset="dft_3d")
atoms = Atoms.from_dict(dat["atoms"])
atoms.write_poscar('POSCAR-JVASP-157789')

In [ ]:
output_features = 1
filename = 'temp/checkpoint_3.pt'
device = "cpu"
if torch.cuda.is_available():
    device = torch.device("cuda")
model = ALIGNN(ALIGNNConfig(name="alignn", output_features=output_features))
model.load_state_dict(torch.load(filename, map_location=device)["model"])
model.eval()
cutoff = 8.0
model = model.to(device)
max_neighbors = 12
jids = ["POSCAR-JVASP-157789"]
original_stdout = sys.stdout
for i in jids:
    atoms = Atoms.from_poscar(i)
    g, lg = Graph.atom_dgl_multigraph(
        atoms, cutoff=float(cutoff), max_neighbors=max_neighbors
    )
    out_data = (
        model([g.to(device), lg.to(device)])
        .detach()
        .cpu()
        .numpy()
        .flatten()
        .tolist()
    )
    print(i, out_data[0])

In [ ]:
!pretrained.py --model_name jv_supercon_tc_alignn --file_format poscar --file_path POSCAR-JVASP-157789